In [1]:
import keras
from keras.layers import Input, Dense, Dropout, LeakyReLU, Softmax, Layer
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
from keras.engine.topology import Layer
from keras.optimizers import Optimizer
from keras.legacy import interfaces
from keras import callbacks
from keras.initializers import VarianceScaling
from sklearn.cluster import KMeans
from keras.optimizers import SGD
from keras.losses import mean_squared_error
from keras.callbacks import EarlyStopping
from keras.initializers import Constant, glorot_normal
from keras.optimizers import Adam

import metrics
from time import time

import numpy as np
import matplotlib.pyplot as plt

import math
import numpy as np

import csv

Using TensorFlow backend.


In [2]:
class ConcreteSelect(Layer):
    def __init__(self, output_dim, start_temp = 10.0, min_temp = 0.1, alpha = 0.99999, **kwargs):
        self.output_dim = output_dim
        self.start_temp = start_temp
        self.min_temp = K.constant(min_temp)
        self.alpha = K.constant(alpha)
        super(ConcreteSelect, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.temp = self.add_weight(name = 'temp', shape = [], initializer = Constant(self.start_temp), trainable = False)
        self.logits = self.add_weight(name = 'logits', shape = [self.output_dim, input_shape[1]], initializer = glorot_normal(), trainable = True)
        super(ConcreteSelect, self).build(input_shape)
        
    def call(self, X, training = None):
        uniform = K.random_uniform(self.logits.shape, K.epsilon(), 1.0)
        gumbel = -K.log(-K.log(uniform))
        temp = K.update(self.temp, K.maximum(self.min_temp, self.temp * self.alpha))
        noisy_logits = (self.logits + gumbel) / temp
        samples = K.softmax(noisy_logits)
        discrete_logits = K.one_hot(K.argmax(self.logits), self.logits.shape[1])

        self.selections = K.in_train_phase(samples, discrete_logits, training)
        Y = K.dot(X, K.transpose(self.selections))
        return Y
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    
class StopperCallback(EarlyStopping):
    
    def __init__(self, mean_max_target = 0.998):
        self.mean_max_target = mean_max_target
        super(StopperCallback, self).__init__(monitor = '', patience = float('inf'), verbose = 1, mode = 'max', baseline = self.mean_max_target)
    
    def on_epoch_begin(self, epoch, logs = None):
        print('mean max of probabilities:', self.get_monitor_value(logs), '- temperature', K.get_value(self.model.get_layer('concrete_select').temp))
        #print( K.get_value(K.max(K.softmax(self.model.get_layer('concrete_select').logits), axis = -1)))
        #print(K.get_value(K.max(self.model.get_layer('concrete_select').selections, axis = -1)))
    
    def get_monitor_value(self, logs):
        monitor_value = K.get_value(K.mean(K.max(K.softmax(self.model.get_layer('concrete_select').logits), axis = -1)))
        return monitor_value


class ConcreteAutoencoderFeatureSelector():
    def __init__(self, K, output_function, num_epochs = 300, batch_size = None, learning_rate = 0.001, start_temp = 10.0, min_temp = 0.1, tryout_limit = 5):

        self.K = K
        self.output_function = output_function
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.start_temp = start_temp
        self.min_temp = min_temp
        self.tryout_limit = tryout_limit
        
    def fit(self, X, Y, val_X = None, val_Y = None):
        assert len(X) == len(Y)
        validation_data = None
        if val_X is not None and val_Y is not None:
            assert len(val_X) == len(val_Y)
            validation_data = (val_X, val_Y)
        
        if self.batch_size is None:
            self.batch_size = max(len(X) // 256, 16)
        
        num_epochs = self.num_epochs
        steps_per_epoch = (len(X) + self.batch_size - 1) // self.batch_size
        
        for i in range(self.tryout_limit):

            K.set_learning_phase(1)

            inputs = Input(shape = X.shape[1:])

            alpha = math.exp(math.log(self.min_temp / self.start_temp) / (num_epochs * steps_per_epoch))
            
            self.concrete_select = ConcreteSelect(self.K, self.start_temp, self.min_temp, alpha, name = 'concrete_select')

            selected_features = self.concrete_select(inputs)

            outputs = self.output_function(selected_features)

            self.model = Model(inputs, outputs)

            self.model.compile(Adam(self.learning_rate), loss = 'mean_squared_error')
            
            print(self.model.summary())
            
            stopper_callback = StopperCallback()
            
            hist = self.model.fit(X, Y, self.batch_size, num_epochs, verbose = 1, callbacks = [stopper_callback], validation_data = validation_data)#, validation_freq = 10)
            
            if K.get_value(K.mean(K.max(K.softmax(self.concrete_select.logits, axis = -1)))) >= stopper_callback.mean_max_target:
                break
            
            num_epochs *= 2
        
        self.probabilities = K.get_value(K.softmax(self.model.get_layer('concrete_select').logits))
        self.indices = K.get_value(K.argmax(self.model.get_layer('concrete_select').logits))
            
        return self
    
    def get_indices(self):
        return K.get_value(K.argmax(self.model.get_layer('concrete_select').logits))
    
    def get_mask(self):
        return K.get_value(K.sum(K.one_hot(K.argmax(self.model.get_layer('concrete_select').logits), self.model.get_layer('concrete_select').logits.shape[1]), axis = 0))
    
    def transform(self, X):
        return X[self.get_indices()]
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        return self.transform(X)
    
    def get_support(self, indices = False):
        return self.get_indices() if indices else self.get_mask()
    
    def get_params(self):
        return self.model

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
    
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

leuk_data=np.loadtxt('simulationdata60.csv',delimiter=',')
print(leuk_data.shape)
x_train=leuk_data;

leuk_label=np.loadtxt('simulationdata60_label.csv',delimiter=',')
print(leuk_label.shape)
y_train=leuk_label-1;

print(x_train)
print(y_train)


(60000, 784)
(10000, 784)
(60, 80)
(60,)
[[-4.3451  -5.7096   4.1448  ...  3.6578   0.2855  -0.13646]
 [-4.0435  -5.0466   3.4547  ... -0.96792 -1.0499  -0.43679]
 [-5.1122  -4.4737   1.1776  ...  1.0954   2.7864  -1.123  ]
 ...
 [ 6.0403   5.6948   1.1209  ...  1.6611   1.741    0.5901 ]
 [ 5.5527   6.6524  -3.454   ...  0.52746  0.23702 -2.0783 ]
 [ 6.1353   5.5938   0.29583 ...  1.6467  -1.2779   1.8621 ]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3.]


In [4]:
def autoencoder_with_kmeans(dims, act='relu', init='glorot_uniform', K=5, num_epochs = 300, batch_size = None, learning_rate = 0.001, start_temp = 10.0, min_temp = 0.1, tryout_limit = 5):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1

    # input
    clustering_input = Input(shape=(dims[-1],), name='clustering_input')
    
    if batch_size is None:
        batch_size = 10 # max(dims[0] // 256, 16)
    
    steps_per_epoch = (dims[0] + batch_size - 1) // batch_size
    x = Input(shape=(dims[0],), name='input')
    alpha = math.exp(math.log(min_temp / start_temp) / (num_epochs * steps_per_epoch))
    concrete_select = ConcreteSelect(K , start_temp, min_temp, alpha, name = 'concrete_select')
    selected_features = concrete_select(x)
    h = (selected_features)

    # internal layers in encoder
    for i in range(n_stacks - 1):
        h = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(h)

    # hidden layer
    h = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(
        h)  # hidden layer, features are extracted from here
    y = h
    # internal layers in decoder
    for i in range(n_stacks - 1, 0, -1):
        y = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(y)
        y = LeakyReLU(0.2)(y)
        y = Dropout(0.1)(y)
        
    # output
    y = Dense(dims[0], kernel_initializer=init, name='decoder_0')(y)

    return Model(inputs=[x, clustering_input], outputs=y, name='AE'), Model(inputs=x, outputs=h,
                                                                            name='encoder'), clustering_input


In [5]:
def loss_wrapper(encoded_X, label_centers, lambd):
    def loss(y_true, y_pred):
        cost_clustering = K.mean(K.square(label_centers - encoded_X), axis=-1)
        cost_reconstruction = K.mean(K.square(y_true - y_pred), axis=-1)

        cost = lambd * cost_clustering + cost_reconstruction
        return cost

    return loss


class DCN(object):
    def __init__(self, dims, n_clusters, lambd=0.5, init='glorot_uniform'):
        super(DCN, self).__init__()
        self.dims = dims
        self.input_dim = dims[0]
        self.n_stacks = len(self.dims) - 1

        self.n_clusters = n_clusters
        self.lambd = lambd
        self.autoencoder, self.encoder, self.clustering_input = autoencoder_with_kmeans(self.dims, init=init)

        self.centers = np.zeros((self.n_clusters, self.dims[-1]))
        self.count = 100 * np.ones(self.n_clusters, dtype=np.int)

    def pretrain(self, x, y=None, optimizer='adam', epochs=200, batch_size=10, save_dir='results/temp'):
        
        print('...Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        csv_logger = callbacks.CSVLogger(save_dir + '/pretrain_log.csv')
        cb = [csv_logger]
        if y is not None:
            class PrintACC(callbacks.Callback):
                def __init__(self, x, y):
                    self.x = x
                    self.y = y
                    super(PrintACC, self).__init__()

                def on_epoch_end(self, epoch, logs=None):
                    if int(epochs / 10) != 0 and epoch % int(epochs / 10) != 0:
                        return
                    feature_model = Model(self.model.input,
                                          self.model.get_layer(
                                              'encoder_%d' % (int(len(self.model.layers) / 2) - 1)).output)
                    features = feature_model.predict(self.x)
                    km = KMeans(n_clusters=len(np.unique(self.y)), n_init=20, n_jobs=4)
                    y_pred = km.fit_predict(features)
                    # print()
                    print(' ' * 8 + '|==>  acc: %.4f,  nmi: %.4f  <==|'
                          % (metrics.acc(self.y, y_pred), metrics.nmi(self.y, y_pred)))

            cb.append(PrintACC(x, y))

        # begin pretraining
        t0 = time()
        temp = []
        for i in range(x.shape[0]):
            t = []
            for k in range(self.dims[-1]):
                t.append(0)
            temp.append(t)
        temp = np.array(temp)
        self.autoencoder.fit([x, temp], x, batch_size=batch_size, epochs=epochs, callbacks=cb)
        print('Pretraining time: %ds' % round(time() - t0))
        self.autoencoder.save_weights(save_dir + '/ae_weights_200.h5')
        print('Pretrained weights are saved to %s/ae_weights_200.h5' % save_dir)
        self.pretrained = True

    def init_centers(self, x, y=None):
        # init self.centers
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(self.encoder.predict(x))
        self.all_pred = kmeans.labels_
        self.centers = kmeans.cluster_centers_

        print('centers-', self.centers)
        if y is not None:
            self.metric(y, self.all_pred)

    def compile(self):
        self.autoencoder.compile(optimizer='adam', loss=loss_wrapper(self.encoder.output, self.clustering_input, 0.5))

    def fit(self, x, y, epoches, batch_size=10, save_dir='./models'):
        m = x_train.shape[0]
        self.count = 100 * np.ones(self.n_clusters, dtype=np.int)
        for step in range(epoches):
            cost = []  # all cost

            for batch_index in range(int(m / batch_size) + 1):
                X_batch = x[batch_index * batch_size:(batch_index + 1) * batch_size, :]

                labels_of_centers = self.centers[self.all_pred[batch_index * batch_size:(batch_index + 1) * batch_size]]

                c1 = self.autoencoder.train_on_batch([X_batch, labels_of_centers], X_batch)
                cost.append(c1)

                reductX = self.encoder.predict(X_batch)

                # update k-means
                self.all_pred[
                batch_index * batch_size:(batch_index + 1) * batch_size], self.centers, self.count = self.batch_km(
                    reductX, self.centers, self.count)

            if step % 10 == 0:
                reductX = self.encoder.predict(x)
                km_model = KMeans(self.n_clusters, init=self.centers)
                self.all_pred = km_model.fit_predict(reductX)
                self.centers = km_model.cluster_centers_

                print('step-', step, ' cost:', np.mean(cost))
                #                print('centers-',self.centers)
                print('count-', self.count)
                self.metric(y, self.all_pred)
        print('saving model to:', save_dir + '/DCN_model_final.h5')
        self.autoencoder.save_weights(save_dir + '/DCN_model_final.h5')

    def batch_km(self, data, center, count):
        """
        Function to perform a KMeans update on a batch of data, center is the
        centroid from last iteration.

        """
        N = data.shape[0]
        K = center.shape[0]

        # update assignment
        idx = np.zeros(N, dtype=np.int)
        for i in range(N):
            dist = np.inf
            ind = 0
            for j in range(K):
                temp_dist = np.linalg.norm(data[i] - center[j])
                if temp_dist < dist:
                    dist = temp_dist
                    ind = j
            idx[i] = ind

        # update centriod
        center_new = center
        for i in range(N):
            c = idx[i]
            count[c] += 1
            eta = 1.0 / count[c]
            center_new[c] = (1 - eta) * center_new[c] + eta * data[i]
        center_new.astype(np.float32)
        return idx, center_new, count

    def get_centers_and_types_of_points(self, reductX):
        distances = np.abs(reductX - self.centers[:, np.newaxis])
        label_types = np.min(np.argmin(distances, axis=0), axis=1)
        labels_of_centers = self.centers[label_types]
        return labels_of_centers, label_types

    def load_weights(self, weights):  # load weights of DEC model
        self.autoencoder.load_weights(weights)

    def extract_features(self, x):
        return self.encoder.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        reductX = self.encoder.predict(x)
        labels_of_centers, label_types = self.get_centers_and_types_of_points(reductX)
        return label_types

    def metric(self, y, y_pred):
        acc = np.round(metrics.acc(y, y_pred), 5)
        nmi = np.round(metrics.nmi(y, y_pred), 5)
        ari = np.round(metrics.ari(y, y_pred), 5)
        print(y_pred)
        print('acc:', acc)
        print('nmi:', nmi)
        print('ari:', ari)

    def get_indices(self):
        print('indices')
        print(K.get_value(K.argmax(self.encoder.get_layer('concrete_select').logits)))
        return K.get_value(K.argmax(self.encoder.get_layer('concrete_select').logits))     
    
    def get_support(self, indices = False):
        return self.get_indices() if indices else self.get_mask()
    
    def get_mask(self):
        print('indices2')
        print(K.get_value(K.sum(K.one_hot(K.argmax(self.encoder.get_layer('concrete_select').logits), self.model.get_layer('concrete_select').logits.shape[1]), axis = 0)))
        return K.get_value(K.sum(K.one_hot(K.argmax(self.encoder.get_layer('concrete_select').logits), self.model.get_layer('concrete_select').logits.shape[1]), axis = 0))
    

In [7]:
dcn = DCN(dims=[x_train.shape[-1], 500, 500, 2000, 10], n_clusters=4, lambd=0.001)
dcn.compile()


ae_weights = 'results/temp/ae_weights_200.h5'
#ae_weights=None

if ae_weights is None:
    pretrain_epochs = 2000
    dcn.pretrain(x=x_train, epochs=pretrain_epochs)
else:
    dcn.autoencoder.load_weights(ae_weights)
dcn.init_centers(x_train, y_train)

dcn.fit(x_train, y_train, epoches=3200, batch_size=40)




centers- [[ 9.47473347e-02  1.44494295e-01  8.40930641e-03 -2.80013680e-01
  -3.86166692e-01 -8.01536143e-02 -1.67951062e-02  1.30814940e-01
  -1.68370873e-01  2.55677104e-02]
 [-1.90093234e-01 -4.79013592e-01  2.75075793e-01  3.15404654e-01
   5.14424515e+00 -1.58418968e-01 -2.48496503e-01  1.27664313e-01
   1.10376418e+00  4.63634193e-01]
 [ 1.83218159e-02  5.66919520e-02 -9.77286696e-03  3.17519569e+00
   1.06598854e-01  2.22254425e-01  5.16950861e-02 -5.70057705e-02
  -4.50560451e-03  1.26445398e-01]
 [-1.59297884e-02 -2.77769536e-01  3.02013516e-01 -4.00174260e-01
   9.46311533e-01 -2.45548129e-01 -8.90003517e-02  1.46945909e-01
   6.14625752e-01  1.57019436e-01]]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 1 1 1 1 1 1 1 1]
acc: 0.96667
nmi: 0.92026
ari: 0.90283


C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead

step- 0  cost: 2.0386338
count- [114 108 116 122]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 1 1 1 1 1 1 1 1]
acc: 0.96667
nmi: 0.92026
ari: 0.90283
step- 10  cost: 2.2694788
count- [250 188 276 346]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 1 1 1 1 1 1 1 1]
acc: 0.96667
nmi: 0.92026
ari: 0.90283


C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arith

step- 20  cost: 1.9253857
count- [390 268 436 566]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 1 1 1 1 1 1 1 1]
acc: 0.96667
nmi: 0.92026
ari: 0.90283
step- 30  cost: 1.968985
count- [530 348 596 786]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3
 0 3 3 3 3 3 3 0 3 3 0 3 3 3 3 1 1 1 1 1 1 1 1]
acc: 0.95
nmi: 0.89575
ari: 0.85789


...

step- 3140  cost: 3.2739278e+21
count- [34277 26256 65857 62470]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699
step- 3150  cost: 2.5946856e+21
count- [34387 26336 66077 62660]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699


C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arith

step- 3160  cost: 1.142165e+22
count- [34497 26416 66297 62850]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699
step- 3170  cost: 7.9725176e+20
count- [34607 26496 66517 63040]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699


C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\cluster\k_means_.py:968: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\Users\admin\anaconda3\envs\pythonProject10\lib\site-packages\sklearn\metrics\cluster\supervised.py:844: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arith

step- 3180  cost: 1.0131121e+22
count- [34717 26576 66737 63230]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699
step- 3190  cost: 8.3010506e+21
count- [34827 26656 66957 63420]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 2 0 0 0 3 3 3 3 3 3 3 3 3
 2 3 3 2 3 3 3 2 3 3 2 3 3 2 3 1 1 1 1 1 1 1 1]
acc: 0.9
nmi: 0.80087
ari: 0.70699
saving model to: ./models/DCN_model_final.h5


In [8]:
#y_pred = dcn.predict(x_test)
#print(y_pred)



In [9]:
#dcn.metric(y_test, y_pred)

In [10]:
dcn.get_support(indices = True)

indices
[1 1 1 1 0]


array([1, 1, 1, 1, 0], dtype=int64)